#### Pre-processing of Kaggle M5 training dataset:


The purpose of this notebook is to format the training data to be able to pre-train my Creme model before going into production. For a product in a shop, I will extract the date, product ID, calendar events and product price.  I can safely say that these informations are available in real time in a production context. I'm also storing the ground truth because we're going to train the model.

For each product, each day, each store I want to obtain the informations:

```
x = {
    'date': '2018-07-15', 
    'id': 'HOBBIES_1_001_CA_1_validation',
    'y': 200, # Ground truth
}
```

In [31]:
import pandas as pd

In [32]:
import pickle

#### Reading training dataset

In [33]:
data = pd.read_csv('./sales_train_validation.csv', dtype = {'id': 'category',
 'item_id': 'category', 'dept_id': 'category', 'cat_id': 'category', 'store_id': 'category',
 'state_id': 'category'})

In [34]:
data.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4


#### Constructing test dataset:

In [35]:
test = pd.DataFrame(data[['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']])
for i in range(1914, 1970):
    test[f'd_{i}'] = 0

In [36]:
test.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1914,d_1915,d_1916,d_1917,...,d_1960,d_1961,d_1962,d_1963,d_1964,d_1965,d_1966,d_1967,d_1968,d_1969
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
test.shape

(30490, 62)

#### Reshape training and testing datasets

In [38]:
metadata = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'] 
sells = [column for column in list(data.columns) if column not in metadata]

In [39]:
data = data.set_index(metadata).stack().reset_index()

In [40]:
test = test.set_index(metadata).stack().reset_index()

#### Distinguish training set from testing set

In [41]:
data['test'] = False
test['test'] = True

In [42]:
data = pd.concat([data, test], sort = False, axis = 'rows')

Here I convert the column d as integer to save memory

In [43]:
data = data.rename(columns = {'level_6': 'd', 0: 'y'})
data['d'] = data['d'].str.split('_').str[1]
data['d'] = data['d'].astype('int16')
data['y'] = data['y'].astype('int32')

In [ ]:
data = data[data['d'] > 1913 - 600]

In [44]:
data.tail()

,id,item_id,dept_id,cat_id,store_id,state_id,d,y,test
1707435,FOODS_3_827_WI_3_validation,FOODS_3_827,FOODS_3,FOODS,WI_3,WI,1965,0,True
1707436,FOODS_3_827_WI_3_validation,FOODS_3_827,FOODS_3,FOODS,WI_3,WI,1966,0,True
1707437,FOODS_3_827_WI_3_validation,FOODS_3_827,FOODS_3,FOODS,WI_3,WI,1967,0,True
1707438,FOODS_3_827_WI_3_validation,FOODS_3_827,FOODS_3,FOODS,WI_3,WI,1968,0,True
1707439,FOODS_3_827_WI_3_validation,FOODS_3_827,FOODS_3,FOODS,WI_3,WI,1969,0,True


#### Extract metadata about the date 

In [45]:
calendar = pd.read_csv('./calendar.csv')

In [46]:
calendar.head()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


Here I convert the column d as integer to save memory

In [47]:
calendar['d'] = calendar['d'].str.split('_').str[1]
calendar['d'] = calendar['d'].astype('int16')

In [48]:
data = pd.merge(left=data, right=calendar, how='left', on=['d'])

In [49]:
train = data[data['test'] == False].copy(deep = True)
train.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,y,test,date,...,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,1,0,False,2011-01-29,...,1,1,2011,NaN,NaN,NaN,NaN,0,0,0
1,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,2,0,False,2011-01-30,...,2,1,2011,NaN,NaN,NaN,NaN,0,0,0
2,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,3,0,False,2011-01-31,...,3,1,2011,NaN,NaN,NaN,NaN,0,0,0
3,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,4,0,False,2011-02-01,...,4,2,2011,NaN,NaN,NaN,NaN,1,1,0
4,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,5,0,False,2011-02-02,...,5,2,2011,NaN,NaN,NaN,NaN,1,0,1


In [50]:
test = data[data['test']].copy(deep = True)
test.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,y,test,date,...,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
58327370,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,1914,0,True,2016-04-25,...,3,4,2016,NaN,NaN,NaN,NaN,0,0,0
58327371,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,1915,0,True,2016-04-26,...,4,4,2016,NaN,NaN,NaN,NaN,0,0,0
58327372,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,1916,0,True,2016-04-27,...,5,4,2016,NaN,NaN,NaN,NaN,0,0,0
58327373,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,1917,0,True,2016-04-28,...,6,4,2016,NaN,NaN,NaN,NaN,0,0,0
58327374,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,1918,0,True,2016-04-29,...,7,4,2016,NaN,NaN,NaN,NaN,0,0,0


#### Sorting the data over time is essential because we want to keep the sales records consistent over time while the model is being trained.

In [51]:
train = train.sort_values('d')

In [52]:
test = test.sort_values('d')

In [69]:
train.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,y,test,date,...,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,week
36466406,HOUSEHOLD_1_208_TX_3_validation,HOUSEHOLD_1_208,HOUSEHOLD_1,HOUSEHOLD,TX_3,TX,801,0,False,2013-04-08,...,4,2013,NaN,NaN,NaN,NaN,1,0,1,15
17946653,HOBBIES_1_242_CA_4_validation,HOBBIES_1_242,HOBBIES_1,HOBBIES,CA_4,CA,801,1,False,2013-04-08,...,4,2013,NaN,NaN,NaN,NaN,1,0,1,15
29158746,FOODS_3_825_TX_1_validation,FOODS_3_825,FOODS_3,FOODS,TX_1,TX,801,0,False,2013-04-08,...,4,2013,NaN,NaN,NaN,NaN,1,0,1,15
38656791,FOODS_2_086_TX_3_validation,FOODS_2_086,FOODS_2,FOODS,TX_3,TX,801,6,False,2013-04-08,...,4,2013,NaN,NaN,NaN,NaN,1,0,1,15
11096200,FOODS_3_527_CA_2_validation,FOODS_3_527,FOODS_3,FOODS,CA_2,CA,801,0,False,2013-04-08,...,4,2013,NaN,NaN,NaN,NaN,1,0,1,15


#### We keep only the columns needed to train the model

In [25]:
columns_to_keep = [
    'date', 
    'id', 
    'y',
]

In [26]:
train = train[columns_to_keep]

In [27]:
test = test[columns_to_keep]

In [28]:
train.to_csv('./train.csv', index=False)

In [29]:
test.to_csv('./test.csv', index=False)


To train my model with the cream library I will iterate on each of the dataset observations. Each observation of the training and testing datasets have the fields of the dictionary below.  I will not use any additional informations from the training and testing datasets I have built.

```
x = {
    'date': '2018-07-15', 
    'id': 'HOBBIES_1_001_CA_1_validation',
    'y': 200, # Ground truth
}
```